In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_KEY

'pcsk_6bYEch_C6MyWm9ftZgVbruvc9Tb4j8Ssd8SWZLhB5DYBTEG5orqgMPxZkh5SQ71Uew6fNe'

In [4]:
from pinecone import Pinecone
# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

In [5]:
# Pinecone 클라이언트를 사용하여 현재 사용 가능한 모든 인덱스의 목록을 반환합니다.
pc.list_indexes()

[
    {
        "name": "wiki",
        "metric": "cosine",
        "host": "wiki-87mf77p.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "test-embedding-3d",
        "metric": "cosine",
        "host": "test-embedding-3d-87mf77p.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 3,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "na

In [6]:
# 사용할 인덱스 이름을 지정합니다.
index_name = "wiki"
# 지정한 이름으로 Pinecone 인덱스를 가져옵니다.
index = pc.Index(index_name)
# 인덱스의 통계 정보를 설명합니다.
index.describe_index_stats()

c:\Users\C113\miniconda3\envs\llmEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki-ns1': {'vector_count': 1080}},
 'total_vector_count': 1080,
 'vector_type': 'dense'}

In [ ]:
from langchain_openai import OpenAIEmbeddings
# OpenAIEmbeddings 객체를 초기화합니다.
# 모델은 "text-embedding-3-small"을 사용하고, API 키는 OPENAI_API_KEY를 사용합니다.
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
# 질문 리스트를 정의합니다.
question = ["U.S television channel G4"]
# 질문을 임베딩하여 벡터로 변환합니다.
embedded_question = embedding.embed_documents(question)
# 임베딩된 질문 벡터를 출력합니다.
print(embedded_question)

[[0.038361743092536926, 0.0569768100976944, 0.003173826029524207, 0.026196883991360664, -0.04045523330569267, -0.008423457853496075, -0.015899188816547394, 0.03872951865196228, 0.017073240131139755, -0.009491419419646263, 0.03465570509433746, 0.04843311756849289, -0.029393697157502174, -0.03505177050828934, 0.006471422500908375, -0.05615638941526413, 0.011464673094451427, -0.033241186290979385, -0.026239320635795593, 0.009427766315639019, -0.013381346128880978, 0.008133482187986374, -0.014753429219126701, 0.013155022636055946, -0.028247935697436333, 0.010219897143542767, -0.0027088031638413668, -0.00046413892414420843, -0.02440044656395912, 0.017964385449886322, 0.05403460934758186, -0.032590508460998535, 0.025631077587604523, -0.05146018788218498, 0.011316148564219475, -0.009222661145031452, 0.00965408980846405, 0.02390536479651928, 0.008225426077842712, 0.01608307659626007, 0.01615380309522152, 0.011839521117508411, 0.01389764528721571, -0.02382049337029457, 0.02913908287882805, -0.0

In [8]:
# Pinecone 인덱스에서 쿼리를 실행합니다.
query_result = index.query(
   namespace="wiki-ns1",  # wiki-ns1 네임스페이스를 지정합니다.
   vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
   top_k=5,  # 상위 5개의 결과를 반환합니다.
   include_vector=False,  # 벡터를 포함하지 않습니다.
   include_metadata=True  # 메타데이터를 포함합니다.
)
# 쿼리 결과에서 각 매치의 ID를 추출하여 리스트로 만듭니다.
result_ids = [r.id for r in query_result.matches]
print(result_ids)  # 결과 ID를 출력합니다.

['007f0fa7-54eb-4bd0-b039-026c8dcb71bf', 'd652f32a-e127-4047-ab2b-ed8b7dc65a06', 'e4d0f93a-52cc-45a2-9136-8318a7600119', 'f5c81d5e-a8dc-45a4-af2f-2fdd1a80c83f', 'f6eb0540-4727-4ba8-be5e-08e626121e75']


In [9]:

# 쿼리 결과에서 각 매치의 점수와 메타데이터를 출력합니다.
for r in query_result.matches:
    print(r.score, r.metadata)

0.660370886 {'chunk_id': 0.0, 'head_chunk': 'This is a list of television programs formerly broadcast by the U.S. cable television channel G4.\n\nSecond iteration\n\nFinal programming\n\nOriginal\n\n Xplay (2004–2013; 2013–2014 (reruns); 2021–2022)\n Att', 'title': 'List of programs broadcast by G4', 'url': 'https://en.wikipedia.org/wiki/List%20of%20programs%20broadcast%20by%20G4', 'wiki_id': '8125821'}
0.55605787 {'chunk_id': 9.0, 'head_chunk': 'Voyagers! (2013)\n\nProgramming blocks \n Expansion Pack (2002–2003)\n G4 Global Gamer (2003–2004)\n Real Time (2003–2005)\n Anime Unleashed (2004–2006) (originated on TechTV and later moved to G4techTV whe', 'title': 'List of programs broadcast by G4', 'url': 'https://en.wikipedia.org/wiki/List%20of%20programs%20broadcast%20by%20G4', 'wiki_id': '8125821'}
0.543539107 {'chunk_id': 14.0, 'head_chunk': 'Morgan Minute - Hosted by Morgan Webb, this segment shows the latest games designed specifically for women.\n Tech Alert - This segment shows th

In [10]:
from langchain_pinecone import PineconeVectorStore
# PineconeVectorStore 객체를 초기화합니다.
# 인덱스는 이전에 정의된 index 객체를 사용하고, 임베딩은 embedding 객체를 사용합니다.
# 텍스트 키는 "full_text"로 설정합니다.
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key="head_chunk")
# 질문에 대한 유사도 검색을 수행합니다.
# query는 question 리스트의 첫 번째 요소를 사용하고, 상위 5개의 결과를 반환합니다.
docs = vector_store.similarity_search(query=question[0], k=5, namespace="wiki-ns1")
print(docs)
# 검색된 문서의 메타데이터를 출력합니다.
for doc in docs:
   print(doc.metadata)

[Document(id='007f0fa7-54eb-4bd0-b039-026c8dcb71bf', metadata={'chunk_id': 0.0, 'title': 'List of programs broadcast by G4', 'url': 'https://en.wikipedia.org/wiki/List%20of%20programs%20broadcast%20by%20G4', 'wiki_id': '8125821'}, page_content='This is a list of television programs formerly broadcast by the U.S. cable television channel G4.\n\nSecond iteration\n\nFinal programming\n\nOriginal\n\n Xplay (2004–2013; 2013–2014 (reruns); 2021–2022)\n Att'), Document(id='d652f32a-e127-4047-ab2b-ed8b7dc65a06', metadata={'chunk_id': 9.0, 'title': 'List of programs broadcast by G4', 'url': 'https://en.wikipedia.org/wiki/List%20of%20programs%20broadcast%20by%20G4', 'wiki_id': '8125821'}, page_content='Voyagers! (2013)\n\nProgramming blocks \n Expansion Pack (2002–2003)\n G4 Global Gamer (2003–2004)\n Real Time (2003–2005)\n Anime Unleashed (2004–2006) (originated on TechTV and later moved to G4techTV whe'), Document(id='e4d0f93a-52cc-45a2-9136-8318a7600119', metadata={'chunk_id': 14.0, 'title':

 - 프롬프트에서 RAG 결과 활용하기

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm=ChatOpenAI(model='gpt-4.1-mini')


In [21]:
chat_template = ChatPromptTemplate.from_messages([
  ('system','''당신은 백과사전 처럼 여러가지 지식을 갖고 있습니다.
   질문에 대해 {context} 활용하여 잘 설명해주세요'''),
  ('user','''{question} 에 대하여 궁금한 지식을 알려주세요.''')
])

In [24]:
context = ""
for doc in docs:
  context += doc.page_content
print(context)

This is a list of television programs formerly broadcast by the U.S. cable television channel G4.

Second iteration

Final programming

Original

 Xplay (2004–2013; 2013–2014 (reruns); 2021–2022)
 AttVoyagers! (2013)

Programming blocks 
 Expansion Pack (2002–2003)
 G4 Global Gamer (2003–2004)
 Real Time (2003–2005)
 Anime Unleashed (2004–2006) (originated on TechTV and later moved to G4techTV wheMorgan Minute - Hosted by Morgan Webb, this segment shows the latest games designed specifically for women.
 Tech Alert - This segment shows the latest and cool gadgets. These are rated on a scale of G4's Late Night Peepshow (2005–2006)
 G4's Training Camp (2005–2006)
 Street Fury (2005–2006)
 Video Game Vixens (2005)
 Wired For Sex (2006–2008)
 The Block (2007–2008)
 Boost Mobile MLG Pro Circuit G4's Proving Ground (2011; 2014; reruns)
 Jump City: Seattle (2011)
 Top 100 Video Games of All Time (2012 TV special)

Acquired 

 10 Play (2003–2004)
 Game Gods (2003–2004)
 Gamer.tv (2003–2004)
 Ga

In [25]:
prompt = chat_template.format_messages(question=question,context=context)
response = llm.invoke(prompt)
print(response.content)

G4는 미국의 케이블 텔레비전 채널로, 주로 비디오 게임, 기술, 그리고 게이머 문화를 다루는 프로그램을 방송했습니다. 원래 2002년에 시작된 이 채널은 비디오 게임과 관련된 다양한 오리지널 프로그램과 여러 가지 acquired 프로그램을 통해 게이머들의 관심을 끌었습니다.

주요 오리지널 프로그램으로는 'Xplay'가 있는데, 2004년부터 2013년까지 방영되었으며 2021년부터 2022년까지 다시 방송되었습니다. 이 프로그램은 게임 리뷰와 게임 산업 뉴스를 제공하는 대표적인 쇼였습니다. 또한 'Attack of the Show!' 같은 인기 프로그램도 있었으나, 위 리스트에는 포함되어 있지 않아 알려드리지 못해 아쉽습니다.

G4는 또한 'Expansion Pack', 'Anime Unleashed', 'G4’s Late Night Peepshow' 같은 다양한 프로그래밍 블록을 제공했는데, 이는 비디오 게임뿐 아니라 기술과 팝 컬처 관련 콘텐츠를 폭넓게 다루었습니다.

취득 프로그램으로는 '10 Play', 'Game Gods', 'Gamer.tv' 등 다양한 게임 관련 프로그램을 방송하였습니다.

그러나, G4는 2014년에 한 차례 폐쇄되었고, 이후 2021년에 잠시 재개되었으나 2022년에 다시 종료된 바 있습니다.

요약하면, G4는 비디오 게임과 기술을 전문적으로 다루던 미국 케이블 채널로, 게이머 및 테크 팬들에게 많은 사랑을 받았던 채널입니다.


In [22]:
# 랭체인 안쓸 때, 처음 했던 방식
from openai import OpenAI

In [23]:
client = OpenAI()
test = client.chat.completions.create(
  model='gpt-4.1-mini',
  messages=[
    {"role":"system","content":'''당신은 백과사전 처럼 여러가지 지식을 갖고 있습니다.
   질문에 대해 활용하여 잘 설명해주세요'''},
    {"role":"user","content":'''{question} 에 대하여 궁금한 지식을 알려주세요.''' }
  ],
  temperature=0.1
)